In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def get_label_from_folder(folder_name):
    index = int(folder_name[6:]) 

    if 1 <= index <= 10:  # Digits 0-9
        return index - 1
    elif 11 <= index <= 36:  # Uppercase A-Z (11 → 'A', ..., 36 → 'Z')
        return index - 1  
    elif 37 <= index <= 62:  # Lowercase a-z (37 → 'a', ..., 62 → 'z')
        return index - 1  
    else:
        return None  


def load_and_preprocess_data(data_dir):
    images = []
    labels = []

    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        if not os.path.isdir(folder_path):
            continue

        label = get_label_from_folder(folder)
        if label is None:
            continue  

        for file in os.listdir(folder_path):
            if file.endswith(".png"):
                img_path = os.path.join(folder_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (28, 28))
                img = img / 255.0  
                images.append(img)
                labels.append(label)  

    X = np.array(images)
    y = np.array(labels)
    return X, y

data_dir = "font/Fnt"  
X, y = load_and_preprocess_data(data_dir)

y = to_categorical(y, num_classes=62)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(62, activation='softmax') 
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True) 

epochs = 100 
batch_size = 64 

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
print("\nEvaluating model performance on the test set...")

y_pred_probabilities = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_probabilities, axis=1)

y_true_labels = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_true_labels, y_pred_labels)
print(f"Test Accuracy: {accuracy:.4f}")

print("\nClassification Report:")

target_names = []
for i in range(10):
    target_names.append(str(i))
for i in range(65, 91):
    target_names.append(chr(i))
for i in range(97, 123):
    target_names.append(chr(i))

print(classification_report(y_true_labels, y_pred_labels, target_names=target_names, zero_division=0))

In [ ]:

print("\nGenerating Confusion Matrix...")

cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(20, 17))
sns.heatmap(cm, annot=False, cmap='Blues',
            xticklabels=target_names, yticklabels=target_names, fmt='d')
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=90, fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
def predict_text_from_image(image_path, model):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Character Segmentation
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours by x-coordinate (left to right)
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    char_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 5 and h > 5:  # Allow smaller characters
            char_img = gray[y:y+h, x:x+w]
            char_img = cv2.resize(char_img, (28, 28))
            char_img = char_img / 255.0
            char_images.append(char_img)

    if not char_images:
        return ""

    char_images = np.array(char_images).reshape(-1, 28, 28, 1)

    # Predict
    predictions = model.predict(char_images)
    predicted_labels = np.argmax(predictions, axis=1)

    # Label to Character Mapping
    characters = ""
    for label in predicted_labels:
        if 0 <= label <= 9:
            characters += str(label)
        elif 10 <= label <= 35:
            characters += chr(label + 55)  # A-Z
        elif 36 <= label <= 61:
            characters += chr(label + 61)  # a-z

    return characters


In [ ]:

image_path = "hw.png"  
predicted_text = predict_text_from_image(image_path, model)
print("Predicted Text:", predicted_text)